# Assignment 3 Group no. 8
### Project members: 
Group NO. 8: YI LI yi9@kth.com CHEN YANG cheyan@kth.com JINRUI LIU jinrui@kth.se

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
# Copy and paste functions from Assignment 1 here that you need for this assignment

# create_normalization
def create_normalization(train_df, normalizationtype = 'zscore'):
    dataf = train_df.copy()
    # old version
    columns = [col for col in list(dataf) if (col != 'ID' and col != 'CLASS')]
    columns = [col for col in columns if (dataf[col].dtype in ['float64', 'float32', 'int'])]
    # new version
    columns = []
    for col in list(dataf):
        if(col not in ['ID', 'CLASS'] and dataf[col].dtype in ['float64', 'float32', 'int']):
            columns.append(col)
    
    normalization = {}
    if normalizationtype == 'zscore':
        for i in columns: 
            mu   = dataf[i].mean()
            delta= dataf[i].std()
            dataf[i] = (dataf[i] - mu)/delta
            normalization[i] = ('zscore',mu,delta) 
    else:                                              
        for i in columns:
            maxx = dataf[i].max()
            minn = dataf[i].min()
            dataf[i] = (dataf[i] - minn)/maxx-minn
            normalization[i] = ('minmax',minn,maxx)          
    return dataf, normalization

# apply_normalization
def apply_normalization(test_df,normalization):                        # this function used on test_dataset
    dataf = test_df.copy()
    columns = [col for col in list(dataf) if (col != 'ID' and col != 'CLASS')]
    columns = [col for col in columns if (dataf[col].dtype in ['float64', 'float32', 'int'])]  
    #print(columns)
    
    # new version
    columns = []
    for col in list(dataf):
        if(col not in ['ID', 'CLASS'] and dataf[col].dtype in ['float64', 'float32', 'int']):
            columns.append(col)
    #print(columns)

    for i in columns:
        if normalization[i][0] == 'zscore':
            #print(normalization[i])
            mu   = normalization[i][1]
            delta= normalization[i][2]
            dataf[i] = (dataf[i] - mu)/delta
        else:
            minn = normalization[i][1]
            maxx = normalization[i][2]
            dataf[i] = (dataf[i] - minn)/(maxx-minn)       # limit the elements within [0,1]
            dataf[i] = [0 if element < 0 else element for element in dataf[i]]
            dataf[i] = [1 if element > 1 else element for element in dataf[i]]
    return dataf   

# create_imputation
def create_imputation(train_df):
    dataf = train_df.copy()
    columns = [col for col in list(dataf) if (col != 'ID' and col != 'CLASS')]
    #print(columns)

    imputa = {}
    for col in list(dataf):
        #print(dataf[col].dtype)
        if(col not in ['ID', 'CLASS'] and dataf[col].dtype in ['float64', 'float32', 'int64', 'int32', 'int']):
            if pd.isnull(dataf[col]).all():          # check if all elements are NaN, replce by 0
                imputa[col] = 0
                dataf[col] = dataf[col].fillna(0)    
            imputa[col]=(dataf[col].mean()) 
            dataf[col] = dataf[col].fillna(imputa[col]) 
            
        elif(col not in ['ID', 'CLASS'] and dataf[col].dtype in ['object', 'category']):
            if pd.isnull(dataf[col]).all():          # check if all elements are NaN, replace by ""
                imputa[col] = ""
                dataf[col] = dataf[col].fillna("") 
                #dataf[i].fillna("",inplace=True)
            imputa[col]=dataf[col].mode()[0] 
            dataf[col] = dataf[col].fillna(imputa[col]) 
            #dataf[i].fillna(mode[0],inplace=True)
    #dataf['CLASS'].astype('category')
    return dataf, imputa

# apply_imputation
def apply_imputation(df_init, imputa):
    dataf = df_init.copy()
    for col in list(dataf):
        if(col not in ['ID','CLASS']): 
            imputa_value = imputa[col]
            dataf[col] = dataf[col].fillna(imputa_value)    # replace NaN of each colume by corresponding Imputation               
    return(dataf)

########################################## Below is for Bayes mainly ########################################################

def create_bins(input_df,nobins,bintype):
    train_df = input_df.copy()
    columns = [col for col in list(train_df) if (col != 'ID' and col != 'CLASS')]
    columns = [col for col in columns if (train_df[col].dtype in ['float64', 'float32', 'int'])]

    # new version
    columns = []
    for col in list(train_df):
        if(col not in ['ID', 'CLASS'] and train_df[col].dtype in ['float64', 'float32', 'int']):
            columns.append(col)

    bin_dict={}
    if bintype == 'equal-size':
        for i in columns:
            train_df[i], binss= pd.qcut(train_df[i], q=nobins,labels=False, retbins=True, duplicates="drop")
            #print(train_df[i].dtypes)
            train_df[i] = pd.Categorical(train_df[i])                   # convert int to categorical data
            bin_temp = binss.tolist()                                   # convert array into list   
            #bin_temp = [round(value,3) for value in bin_temp]          # round to 2 decimal
            bin_temp[0]=-np.inf                                         # replace the first and last elements
            bin_temp[-1]=np.inf
            bin_dict[i]=bin_temp                                        # map into dict
    else:
        for i in columns:
            train_df[i], binss= pd.cut(train_df[i], nobins,labels=False, retbins=True, duplicates="drop")
            #print(train_df[i].dtypes)
            train_df[i] = pd.Categorical(train_df[i])                   # convert int to categorical data
            bin_temp = binss.tolist()                                   # convert array into list   
            #bin_temp = [round(value,3) for value in bin_temp]          # round to 2 decimal
            bin_temp[0]=-np.inf                                         # replace the first and last elements
            bin_temp[-1]=np.inf
            bin_dict[i]=bin_temp                                        # map into dict      
    train_df['CLASS'] = train_df['CLASS'].astype('category')
    return train_df,bin_dict

def apply_bins(input_df, binning):
    test_df = input_df.copy()
    #columns = [col for col in list(test_df) if (col != ['ID', 'CLASS'])]
    columns = [col for col in list(test_df) if (col != 'ID' and col != 'CLASS')]
    columns = [col for col in columns if (test_df[col].dtype in ['float64', 'float32', 'int'])]
    
    # new version
    columns = []
    for col in list(test_df):
        if(col not in ['ID', 'CLASS'] and test_df[col].dtype in ['float64', 'float32', 'int']):
            columns.append(col)
    #print(columns)

    for i in columns:
        binss = binning[i]                                         # get the corresponding bins
        labelss = list(range(0,len(binss)-1))                      # labels + 1 = bin values
        test_df[i] = pd.cut(test_df[i], bins=binss, labels=labelss, duplicates="drop")
        #print(test_df[i].dtypes)                                  # .cut automatically gives categorical data
        #test_df[i] = pd.Categorical(test_df[i])                    
    return test_df
##################################### Below is for testing ####################################################

def brier_score(df, correctl):
    m,n = df.shape[0],df.shape[1]                                      #  m data with n categories
    score = 0
    for i in range(m):                              
        idx = np.where(df.columns==correctl[i])[0]                # return columns index of corresponding category
        corr_binary = [1 if j == idx else 0 for j in range(n)]         # create binary list for this category
        temp = sum((corr_binary - df.iloc[i,:])**2)
        score = score + temp
    score = score/m
    return score

def accuracy(predictions,correctlabels):
    n = predictions.shape[0]
    correct = 0
    for i in range(n):
        df1 = predictions.iloc[i]
        pred_label = df1.idxmax()
        if (pred_label == correctlabels[i]):
            correct += 1
    rate = correct/n
    return(rate)

def label(predictions,correctlabels):
    classes = list(predictions)      # return A,B,C
    label_dict={}
    for c in classes:
        label_dict[c] = [1 if temp == c else 0 for temp in correctlabels]
    return(label_dict)


def binary_auc_dict(prediction, label):       # inputs are one column of prediction and one column of label
    score = np.unique(prediction)             # this .unique directly sort the values
    score = score[::-1]
    sco_dict = dict.fromkeys(score,[0,0])
        
    for i in range(len(label)):
        if label[i] == 1:                     # positive
            temp = sco_dict[prediction[i]][0] + 1
            sco_dict[prediction[i]] = [temp, sco_dict[prediction[i]][1]]
        else:                                 # negtive
            temp = sco_dict[prediction[i]][1] + 1
            sco_dict[prediction[i]] = [sco_dict[prediction[i]][0], temp]
    return sco_dict


def binary_auc(tp,fp):
    auc = 0
    cov_tp = 0
    n = len(tp)
    if sum(tp)==0:
        auc = 0
    else:    
        for i in range(n):
            if fp[i] == 0:
                cov_tp +=tp[i]
            elif tp[i] == 0:
                auc += (cov_tp/sum(tp))*(fp[i]/sum(fp))
            else:
                auc += (cov_tp/sum(tp))*(fp[i]/sum(fp)) + (tp[i]/sum(tp))*(fp[i]/sum(fp))/2
                cov_tp += tp[i]
    return(auc)


def weighted_auc(auc_dict, correctlabels):
    num = len(correctlabels)
    labels = [1,2,3,4,5,6,7]
    if type(correctlabels) == list:                  # here we assume the testing label is a 'Pandas column' OR a 'list'
        correctlabels = correctlabels
    else:
        correctlabels = correctlabels.tolist()
    frequency = {x:correctlabels.count(x)/num for x in correctlabels}      
    final_auc = int(0)       
    #print("Weights:",frequency)                     # the key types of two dictionaries should be consistent
    #dict_keys = auc_dict.keys()
    #freq_keys = frequency.keys()
    #print(dict_keys)
    #print(freq_keys)                       
    #print(frequency)
    for classes in list(auc_dict):
        temp = frequency[classes]*auc_dict[classes][0]
        final_auc += temp
    return final_auc


def auc(predictions,correctlabels):
    aaa = label(predictions,correctlabels)
    temp = 0
    auc_dict={}
    for col in predictions.columns:  
        bbb = binary_auc_dict(predictions[col],aaa[col])
        bbb = pd.DataFrame.from_dict(bbb, orient='index')
        bbb.index=range(len(bbb))
        pos = bbb.iloc[:,0]
        neg = bbb.iloc[:,1]
        temp = binary_auc(pos,neg)
        auc_dict[col] = [temp]
    #print(" Areas:",auc_dict)
    auc = weighted_auc(auc_dict,correctlabels)
    return(auc)

## 1. Define the class DecisionTree

In [3]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
############################################################################################################################
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# Output from fit:
# nothing      

# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
#
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
#
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
#
# Hint 5: The best feature found (with lowest residual information content) will be used to split the training
#         instances, and each sub-group will be used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
#
# Hint 6: You may make divide_and_conquer return not only a list of nodes, but also a current_node_no; 
#         by this, each subsequent call to divide_and_conquer for each subset of instances, i.e. for each feature value, 
#         could use current_node_no as a starting point.
#         If you e.g. make the following call:
#
#         current_node_no, node_list = divide_and_conquer(current_node_no, ...)
#
#         then the returned value in current_node_no can be used in the next call to divide_and_conquer.
#         Node that node_list will contain an arbitrary number of tuples, each element corresponding to a node together 
#         with a node number. The first element in the list will have the same number as current_node_no when the call 
#         was made and the last element will have a number one less than current_node_no when returned, e.g., if there is
#         only one (leaf) node in the returned list, then current_node_no will only be incremented by one through the above call.
#         
# Hint 7: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array

# The result of applying this function should be:
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 

# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
############################################################################################################################
class DecisionTree:
    def __init__(self):
        self.binning=None
        self.imputation=None
        self.labels=None
        self.model=None
    
    def information_content(self, df):
        numEntries = df.shape[0]
        label_count = {}
        for index, feat in df.iterrows():
            currentLabel = int(feat["CLASS"])
            if currentLabel not in label_count:
                label_count[currentLabel] = 0
            label_count[currentLabel] += 1
        information_content = 0.0
        for label in label_count:
            prob = float(label_count[label]) / numEntries
            information_content -= prob * np.log2(prob)
        #print(information_content[1])
        return information_content
    
    
    def get_vector(self, df):
        groupSize = df.groupby("CLASS").size()
        #groupSize = self.get_group_size(df)
        vec = []
        for label in self.labels:
            vec.append(groupSize[label])
        return vec
    
    def dfSplit(self, df, feature, value):
        retdataset = df[df[feature] == value].copy()
        del retdataset[feature]
        return retdataset
    
    def BestSplitFeature(self, df, fealist):
        baseEntroy = self.information_content(df)
        baseInfoGain = 0.0
        bestfeature = fealist[0]
        for feature in fealist:
            features = df[feature]
            uniqueVals = set(features)
            newEntrypoy = 0.0
            for value in uniqueVals:
                subdataset = self.dfSplit(df, feature ,value)
                prob = len(df[df[feature]==value])/float(len(df))
                newEntrypoy += prob*self.information_content(subdataset)
            infogain =  newEntrypoy - baseEntroy
            if (infogain > baseInfoGain):
                baseInfoGain = infogain
                bestfeature = feature
        return bestfeature 
    
    def divide_and_conquer(self, df, fealist, groupSize, min_sample_split, nodeno):
        labelList = list(df["CLASS"].values)
        vec = []
        for label in self.labels:
            vec.append(groupSize[label])
        if labelList.count(labelList[0]) == len(labelList):
            return (nodeno, "leaf", vec)
        if fealist is None or len(fealist) == 0:
            return (nodeno, "leaf", vec)
        bestFeature = self.BestSplitFeature(df, fealist)
        dit = {}
        uniqueVals = df[bestFeature].unique()
        feas = fealist.copy()
        feas.remove(bestFeature)       
        for value in uniqueVals:
            sptData = self.dfSplit(df, bestFeature, value)
            self.nextNode += 1
            dit[value] = self.nextNode
            if sptData.shape[0] >= min_sample_split:
                group_size_temp = sptData.groupby("CLASS").size()
                subtree = self.divide_and_conquer(sptData, feas, group_size_temp, min_sample_split, self.nextNode)
            else:
                subtree = (self.nextNode, "leaf", self.get_vector(sptData))
            self.model.append(subtree)
        return (nodeno, bestFeature, dit)
    
    
    def fit(self, input_df, nobins = 10, bintype = "equal-width", min_samples_split = 5):    #Assign values
        df = input_df.copy()
        df, self.binning = create_bins(df, nobins, bintype)
        df, self.imputation = create_imputation(df)
        classes = df['CLASS'].astype('category')     # this is already done in create_functions
        self.class_priors = dict(classes.value_counts(normalize = True))
        self.labels = classes.cat.categories  
        self.model = []
        self.nextNode = 0   
        
        columns = [col for col in list(df) if (col != 'ID' and col != 'CLASS')]
        feature_list = []
        for feature in columns:
                feature_list.append(feature)
        class_size = df.groupby("CLASS").size()      # class count
        class_freq = class_size/sum(class_size)      # class relative frequency
        

        #current_node_no, node_list = divide_and_conquer(current_node_no, ...)
        # I directly re-visit the current node no inside the divide_and_conquer function
        root = self.divide_and_conquer(df, feature_list, class_size, min_samples_split, 0)  
        #print("root:",root)
        self.model.append(root)
        self.model = sorted(self.model, key=lambda x:x[0])
        

###########################################################################################################################
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root
############################################################################################################################
    def make_prediction(self, nodeno, row):
        node = self.model[nodeno]
        if node[1] == "leaf":
            return (node[2])
        else:
            if row[node[1]] in node[2]:
                return self.make_prediction(node[2][row[node[1]]], row)
            else:
                keys = node[2].keys()
                keys = sorted(keys)
                if row[node[1]] < keys[0]:
                    return self.make_prediction(node[2][keys[0]], row)
                for key in keys:
                    if key > row[node[1]]:
                        return self.make_prediction(node[2][key], row)
                return self.make_prediction(node[2][keys[-1]], row)
                

    def predict(self, df):
        columns = [col for col in list(df) if (col != 'ID' and col != 'CLASS')]    
        df = df[columns]
        df = apply_imputation(df, self.imputation)
        df = apply_bins(df, self.binning)
        labels = []
        for index, row in df.iterrows():
            predictions = self.make_prediction(0, row)
            labels.append(predictions)
        return pd.DataFrame(labels, columns=list(map(int, self.labels)))   


In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results


Training time (5, 'equal-width', 3): 3.84 s.
Testing time (5, 'equal-width', 3): 0.05 s.
Training time (5, 'equal-width', 5): 2.91 s.
Testing time (5, 'equal-width', 5): 0.04 s.
Training time (5, 'equal-width', 10): 1.75 s.
Testing time (5, 'equal-width', 10): 0.04 s.
Training time (5, 'equal-size', 3): 4.32 s.
Testing time (5, 'equal-size', 3): 0.04 s.
Training time (5, 'equal-size', 5): 2.35 s.
Testing time (5, 'equal-size', 5): 0.04 s.
Training time (5, 'equal-size', 10): 1.48 s.
Testing time (5, 'equal-size', 10): 0.04 s.
Training time (10, 'equal-width', 3): 4.15 s.
Testing time (10, 'equal-width', 3): 0.04 s.
Training time (10, 'equal-width', 5): 2.60 s.
Testing time (10, 'equal-width', 5): 0.05 s.
Training time (10, 'equal-width', 10): 2.86 s.
Testing time (10, 'equal-width', 10): 0.05 s.
Training time (10, 'equal-size', 3): 3.96 s.
Testing time (10, 'equal-size', 3): 0.07 s.
Training time (10, 'equal-size', 5): 3.39 s.
Testing time (10, 'equal-size', 5): 0.05 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.560748     8.869159  0.723693
               5   0.607477     9.158879  0.748862
               10  0.626168    17.607477  0.746434
   equal-size  3   0.588785     3.121495  0.743689
               5   0.635514     4.588785  0.755014
               10  0.542056    10.728972  0.697604
10 equal-width 3   0.644860     4.000000  0.809237
               5   0.598131     5.317757  0.790216
               10  0.579439     8.345794  0.748116
   equal-size  3   0.607477     1.429907  0.741858
               5   0.570093     1.831776  0.731188
               10  0.579439     3.271028  0.735305

In [5]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 0.99
Brier score on training set: 3.68


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class DecisionForest

In [6]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
class DecisionForest:
    def __init__(self):
        self.binning=None
        self.imputation=None
        self.labels=None
        self.model=None
        
    def information_content(self, df):
        numEntries = df.shape[0]
        labelCounts = {}
        for index, feat in df.iterrows():
            currentLabel = int(feat["CLASS"])
            if currentLabel not in labelCounts:
                labelCounts[currentLabel] = 0
            labelCounts[currentLabel] += 1
        information_content = 0.0
        for label in labelCounts:
            prob = float(labelCounts[label]) / numEntries
            information_content -= prob * np.log2(prob)
        return information_content
    
    def splitDataFrame(self, df, feature, value):
        retdataset = df[df[feature] == value].copy()
        del retdataset[feature]
        return retdataset
    
    def BestSplitFeature(self, df, features):
        baseEntroy = self.information_content(df)
        baseInfoGain = 0.0
        bestfeature = features[0]
        for feature in features:
            featlist = df[feature]
            uniqueVals = set(featlist)
            newEntrypoy = 0.0
            for value in uniqueVals:
                subdataset = self.splitDataFrame(df, feature ,value)
                prob = len(df[df[feature]==value])/float(len(df))
                newEntrypoy += prob*self.information_content(subdataset)
            infogain =  newEntrypoy - baseEntroy
            if (infogain > baseInfoGain):
                baseInfoGain = infogain
                bestfeature = feature
        return bestfeature
    
    
    def get_vector(self, df):
        groupSize = df.groupby("CLASS").size()
        vec = []
        for label in self.labels:
            vec.append(groupSize[label])
        return vec
    
    def divide_and_conquer(self, df, features, groupSize, min_sample_split, nodeno):
        labelList = list(df["CLASS"].values)
        vec = []
        for label in self.labels:
            vec.append(groupSize[label])
        if labelList.count(labelList[0]) == len(labelList):
            return (nodeno, "leaf", vec)
        if features is None or len(features) == 0:
            return (nodeno, "leaf", vec)
        bestFeature = self.BestSplitFeature(df, features)
        dit = {}
        uniqueVals = df[bestFeature].unique()
        feas = features.copy()
        feas.remove(bestFeature)        
        for value in uniqueVals:
            sptData = self.splitDataFrame(df, bestFeature, value)
            self.nextNode += 1
            dit[value] = self.nextNode
            if sptData.shape[0] >= min_sample_split:    
                group_size_temp = sptData.groupby("CLASS").size()
                subtree = self.divide_and_conquer(sptData, feas, group_size_temp, min_sample_split, self.nextNode)
            else:
                subtree = (self.nextNode, "leaf", self.get_vector(sptData))
            self.model.append(subtree)
        return (nodeno, bestFeature, dit)


    def fit(self, df, nobins=10, bintype="equal-width", min_samples_split=5, random_features=2, notrees=10):

        df, self.binning = create_bins(df, nobins, bintype)
        df, self.imputation = create_imputation(df)
        classes = df['CLASS'].astype('category')     # this is already done in create_functions
        self.labels = classes.cat.categories  
        self.trees = [] 
        
        columns = [col for col in list(df) if (col != 'ID' and col != 'CLASS')]
        feature_list = []
        for feature in columns:
                feature_list.append(feature)
        class_size = df.groupby("CLASS").size()      # class count
        class_freq = class_size/sum(class_size)      # class relative frequency
             
        for i in range(notrees):
            if random_features == 0:
                self.model = []
                self.nextNode = 0
                root = self.divide_and_conquer(df, feature_list, class_size, min_samples_split, 0)
                self.model.append(root)
                self.model = sorted(self.model, key=lambda x:x[0])
#               print(model)
                self.trees.append(self.model.copy())
            else:
                feas = np.random.choice(feature_list, random_features, replace=False).tolist()
                self.model = []
                self.nextNode = 0
                root = self.divide_and_conquer(df, feas, class_size, min_samples_split, 0) 
                self.model.append(root)
                self.model = sorted(self.model, key=lambda x:x[0])
                self.trees.append(self.model.copy())
                
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities
    def make_prediction(self, nodeno, row, tree):
        node = tree[nodeno]
        if node[1] == "leaf":
            return (node[2])
        else:
            if row[node[1]] in node[2]:
                return self.make_prediction(node[2][row[node[1]]], row, tree)
            else:
                keys = node[2].keys()
                keys = sorted(keys)
                if row[node[1]] < keys[0]:
                    return self.make_prediction(node[2][keys[0]], row, tree)
                for key in keys:
                    if key > row[node[1]]:
                        return self.make_prediction(node[2][key], row, tree)
                return self.make_prediction(node[2][keys[-1]], row, tree)

    def predict(self, df):
        columns = [col for col in list(df) if (col != 'ID' and col != 'CLASS')]    
        df = df[columns]
        df = apply_imputation(df, self.imputation)
        df = apply_bins(df, self.binning)
        labels = []
        for index, row in df.iterrows():
            curs = []
            for tree in self.trees:
                probs = self.make_prediction(0, row, tree)
                curs.append(probs)
            curs = np.array(curs)
            curs = np.mean(curs, axis=0)
            labels.append(curs)
        return pd.DataFrame(labels, columns=list(map(int, self.labels)))


In [7]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 1.16 s.
Testing time (1, 1): 0.06 s.
Training time (1, 2): 4.63 s.
Testing time (1, 2): 0.15 s.
Training time (1, 5): 20.49 s.
Testing time (1, 5): 0.13 s.
Training time (2, 1): 1.00 s.
Testing time (2, 1): 0.05 s.
Training time (2, 2): 3.79 s.
Testing time (2, 2): 0.08 s.
Training time (2, 5): 22.27 s.
Testing time (2, 5): 0.11 s.
Training time (5, 1): 1.19 s.
Testing time (5, 1): 0.08 s.
Training time (5, 2): 3.50 s.
Testing time (5, 2): 0.08 s.
Training time (5, 5): 14.76 s.
Testing time (5, 5): 0.10 s.


Accuracy  Brier score       AUC
1 1  0.616822   399.482523  0.681875
  2  0.607477    61.677944  0.713227
  5  0.682243     3.428972  0.828197
2 1  0.542056   698.582243  0.656358
  2  0.588785   101.406075  0.707183
  5  0.691589     3.412243  0.838170
5 1  0.626168   562.335047  0.688434
  2  0.626168   109.784953  0.625709
  5  0.644860     4.431682  0.814985

In [8]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.65
AUC on training set: 0.84
Brier score on training set: 57.70


### Comment on assumptions, things that do not work properly, etc.
We implemented both Decision Tree and Decision Forest for this assignment, the algorithm is quite complex for us, and we have no more time to improve the accuracy. 

Worth to mention that, we adjusted every assignment with the given datasets on Canvas, and made sure the code can successfully run before submission, which can be referred to out github: https://github.com/liyi201809/Programming-for-Data-Science-ID2214.

According to the feedback on the previous assignment, I think our code meets errors mainly because it was tested by some other datasets so some data type conversion cannot work well anymore. I hope this will not influence our grades too much since all of us put a lot efforts into these assignments, and the general algorithms/implementations are in a correct way.

Thanks for your reviewing, and we are willing to discuss it further!